In [7]:
# %matplotlib inline
import matplotlib
matplotlib.use('agg')
from matplotlib import pyplot as plt

import pickle
import os
import tensorflow as tf
import PIL
import IPython
import cv2
import time

from metrab_utils import *

def showarray(a, fmt='jpeg'):
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

model_fold = '/disk/apps/metrabs/models'

models = os.listdir(model_fold)
print(models)

['metrabs_rn18_y4', 'metrabs_eff2l_y4_360', 'metrabs_eff2s_y4', 'metrabs_mob3s_y4t', 'server.log', 'metrabs_mob3s_y4', 'metrabs_eff2l_y4']


In [8]:
#mod = 'metrabs_mob3s_y4t' #10fps on rtx2070s
mod = 'metrabs_rn18_y4' #9fps on rtx2070s

t = time.time()
model = tf.saved_model.load(os.path.join(model_fold, mod))
t = time.time() - t
print('load took %.2fs'%t)

2021-12-13 11:54:11.193370: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-13 11:54:11.205770: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-13 11:54:11.206852: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-13 11:54:11.208672: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

load took 23.04s


In [2]:
cam = cv2.VideoCapture(0)

try:
    while(True):
        t1 = time.time()
        
        # Capture frame-by-frame
        ret, frame = cam.read()

        # Convert the image from OpenCV BGR format to matplotlib RGB format
        # to display the image
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        pred = model.detect_poses(frame)
        
        t2 = time.time()
        
        showarray(frame)

        print("%f FPS, %.2fms" % (1/(t2-t1), (t2-t1)*1000))
        # Display the frame until new frame is available
        IPython.display.clear_output(wait=True)
except KeyboardInterrupt:
    print("Stream stopped")
cam.release()

NameError: name 'model' is not defined

In [9]:
img_local = cv2.imread('cpp/image.png')
pred = model.detect_poses(img_local)
pred['poses3d']

2021-12-13 11:54:44.744536: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202
2021-12-13 11:54:45.720375: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x16ff8b80


<tf.Tensor: shape=(1, 122, 3), dtype=float32, numpy=
array([[[  89.61586 ,  -14.519338, 2018.1293  ],
        [ -31.130798,  -20.164759, 2028.8268  ],
        [  41.907276, -210.75435 , 2058.418   ],
        [ 160.08533 ,  356.83694 , 2017.6292  ],
        [-151.5985  ,  338.24725 , 2030.4164  ],
        [  47.56598 , -336.89554 , 2063.8628  ],
        [ 185.7382  ,  747.76544 , 2101.3433  ],
        [-193.17998 ,  741.67267 , 2081.017   ],
        [  47.175434, -386.92535 , 2043.741   ],
        [ 224.9565  ,  815.6357  , 1996.0869  ],
        [-260.67203 ,  790.3314  , 1979.6912  ],
        [  57.950592, -604.31995 , 2074.6948  ],
        [ 124.30111 , -504.09976 , 2067.601   ],
        [ -16.165485, -513.8124  , 2078.2776  ],
        [  55.55847 , -658.5768  , 2027.5123  ],
        [ 209.05122 , -543.34924 , 2064.794   ],
        [-101.15608 , -560.0211  , 2089.1216  ],
        [ 421.7885  , -528.7523  , 2048.2722  ],
        [-314.6938  , -551.3535  , 2081.0625  ],
        [ 627.71